# 2022-1-27
# 本代码计算ceof

In [8]:
import os
import sys
import xarray as xr
import numpy as np
import scipy.linalg
module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *
import cartopy.crs as ccrs

In [9]:
path0 = "/home/sun/data/ceof-lxl/2022-1-27/"
f1    = xr.open_dataset(path0+"PC1_neg_oned_hibert_sf.nc")
f2    = xr.open_dataset(path0+"PC1_pos_oned_hibert_sf.nc")
f3    = xr.open_dataset(path0+"PC2_neg_oned_hibert_sf.nc")
f4    = xr.open_dataset(path0+"PC2_pos_oned_hibert_sf.nc")


In [13]:
matrix1  =  f1.ZR.data + 1J*f1.ZI.data
matrix2  =  f2.ZR.data + 1J*f2.ZI.data
matrix3  =  f3.ZR.data + 1J*f3.ZI.data
matrix4  =  f4.ZR.data + 1J*f4.ZI.data

s1       =  1/255 * np.dot(matrix1,matrix1.T.conjugate())
s2       =  1/255 * np.dot(matrix2,matrix2.T.conjugate())
s3       =  1/255 * np.dot(matrix3,matrix3.T.conjugate())
s4       =  1/255 * np.dot(matrix4,matrix4.T.conjugate())

values1,vectors1 = scipy.linalg.eigh(s1,np.identity(255))
values2,vectors2 = scipy.linalg.eigh(s2,np.identity(255))
values3,vectors3 = scipy.linalg.eigh(s3,np.identity(255))
values4,vectors4 = scipy.linalg.eigh(s4,np.identity(306))

space_matrix1    =  np.dot(vectors1.T.conjugate(),matrix1)
space_matrix2    =  np.dot(vectors2.T.conjugate(),matrix2)
space_matrix3    =  np.dot(vectors3.T.conjugate(),matrix3)
space_matrix4    =  np.dot(vectors4.T.conjugate(),matrix4)

v1  =  np.dot(matrix1.T.conjugate(),vectors1)/(255 ** 0.5)
v2  =  np.dot(matrix2.T.conjugate(),vectors2)/(255 ** 0.5)
v3  =  np.dot(matrix3.T.conjugate(),vectors3)/(255 ** 0.5)
v4  =  np.dot(matrix4.T.conjugate(),vectors4)/(306 ** 0.5)

matrix_eigen1  =  np.zeros((255,255),dtype=complex)
matrix_eigen2  =  np.zeros((255,255),dtype=complex)
matrix_eigen3  =  np.zeros((255,255),dtype=complex)
matrix_eigen4  =  np.zeros((306,306),dtype=complex)

for i in range(255):
    matrix_eigen1[i,i]  =  values1[i]
    matrix_eigen2[i,i]  =  values2[i]
    matrix_eigen3[i,i]  =  values3[i]
    

for i in range(306):
    matrix_eigen4[i,i]  =  values4[i]

t1   =  matrix_eigen1.copy()
tt1  =  np.dot(matrix1,v1)
t2   =  matrix_eigen2.copy()
tt2  =  np.dot(matrix2,v2)
t3   =  matrix_eigen3.copy()
tt3  =  np.dot(matrix3,v3)
t4   =  matrix_eigen4.copy()
tt4  =  np.dot(matrix4,v4)

for i in range(255):
    t1[i,i]  =  tt1[i,i] / (255**0.5)*matrix_eigen1[i,i]
    t2[i,i]  =  tt2[i,i] / (255**0.5)*matrix_eigen2[i,i]
    t3[i,i]  =  tt3[i,i] / (255**0.5)*matrix_eigen3[i,i]
    

for i in range(306):
    t4[i,i]  =  tt4[i,i] / (306**0.5)*matrix_eigen4[i,i]

ncfile  =  xr.Dataset(
    {
        "v1_real": (["axis1","axis2"], v1.real),
        "v1_imag": (["axis1","axis2"], v1.imag),
        "t1_real": (["axis2","axis2"], t1.real),
        "t1_imag": (["axis2","axis2"], t1.imag),
        "v2_real": (["axis1","axis2"], v2.real),
        "v2_imag": (["axis1","axis2"], v2.imag),
        "t2_real": (["axis2","axis2"], t2.real),
        "t2_imag": (["axis2","axis2"], t2.imag),
        "v3_real": (["axis1","axis2"], v3.real),
        "v3_imag": (["axis1","axis2"], v3.imag),
        "t3_real": (["axis2","axis2"], t3.real),
        "t3_imag": (["axis2","axis2"], t3.imag),
        "v4_real": (["axis3","axis4"], v4.real),
        "v4_imag": (["axis3","axis4"], v4.imag),
        "t4_real": (["axis4","axis4"], t4.real),
        "t4_imag": (["axis4","axis4"], t4.imag),
    },
    coords={
        "axis1": (["axis1"], np.linspace(1,v1.shape[0],v1.shape[0])),
        "axis2": (["axis2"], np.linspace(1,v1.shape[1],v1.shape[1])),
        "axis3": (["axis3"], np.linspace(1,v4.shape[0],v4.shape[0])),
        "axis4": (["axis4"], np.linspace(1,v4.shape[1],v4.shape[1])),
    },
)
ncfile.attrs["desctiption"]  =  "for lxl ceof 2022-1-27"
ncfile.to_netcdf("/home/sun/data/ceof_out_lxl_2022-1-27.nc")